# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import pickle
import warnings
warnings.filterwarnings('ignore')

nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)

X = df['message']
y = df.drop(['message', 'genre', 'original', 'child_alone'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z]", " ", str(text))
    tokens = word_tokenize(str(text))
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        #if len(clean_tok) <= 30: 
        #most words are less than 30 characters and there are some typos without spaces to remove using this
        #    clean_tokens.append(clean_tok)
        #else: 
        #    pass
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#vect = CountVectorizer(tokenizer=tokenize)
#tfidf = TfidfTransformer()
#clf = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
            ('vect', CountVectorizer(stop_words={'english'}, tokenizer=tokenize))
             ,('tfidf', TfidfTransformer())
             ,('clf', RandomForestClassifier())
            ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#split data into train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
# train classifier
#X_train_counts = vect.fit_transform(X_train)
#X_train_tfidf = tfidf.fit_transform(X_train_counts)
#clf.fit(X_train_tfidf, y_train)

pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words={'english'},
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [6]:
#pipeline.named_steps["vect"].get_params()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data

y_pred = pipeline.predict(X_test)

In [8]:
for i in range(len(y_test.columns)):
    print(y_test.columns[i],metrics.classification_report(y_test.iloc[:,i], y_pred[:,i]))

related              precision    recall  f1-score   support

          0       0.57      0.48      0.52      1544
          1       0.85      0.89      0.87      5010

avg / total       0.78      0.79      0.79      6554

request              precision    recall  f1-score   support

          0       0.89      0.99      0.94      5464
          1       0.85      0.39      0.53      1090

avg / total       0.88      0.89      0.87      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.71      0.91      0.80      3879
          1       0.78      0.45      0.57      2675

avg / total       0.74      0.72      0.71      6554

medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0

In [9]:
print("precision    recall  f1-score   support")
print(metrics.precision_recall_fscore_support(y_test, y_pred, average='macro'))

precision    recall  f1-score   support
(0.60248310887186263, 0.12644385046070694, 0.17312812190658231, None)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {'vect__min_df':[0.01, 0.05]
              ,'clf__max_depth': [10, 100, 200]}

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
#train and test model
cv.fit(X_train, y_train)
y_pred2 = cv.predict(X_test)


In [12]:
#view best parameters
cv.best_params_

{'clf__max_depth': 200, 'vect__min_df': 0.01}

In [13]:
#view all parameters and mean test scores, std of test scores
for p,m,std in zip(cv.cv_results_['params'], cv.cv_results_['mean_test_score'], cv.cv_results_['std_test_score']):
    print(p,m,std)

{'clf__max_depth': 10, 'vect__min_df': 0.01} 0.197049847406 0.00490976865234
{'clf__max_depth': 10, 'vect__min_df': 0.05} 0.206307222787 0.000445903442899
{'clf__max_depth': 100, 'vect__min_df': 0.01} 0.25483214649 0.00487332946551
{'clf__max_depth': 100, 'vect__min_df': 0.05} 0.212919633774 0.0021516709618
{'clf__max_depth': 200, 'vect__min_df': 0.01} 0.261190233978 0.00278885495045
{'clf__max_depth': 200, 'vect__min_df': 0.05} 0.215920651068 0.00277250198743


In [14]:
#check out performance of model with best parameters
for i in range(len(y_test.columns)):
    print(y_test.columns[i],metrics.classification_report(y_test.iloc[:,i], y_pred2[:,i]))

related              precision    recall  f1-score   support

          0       0.54      0.43      0.48      1544
          1       0.84      0.88      0.86      5010

avg / total       0.77      0.78      0.77      6554

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      5464
          1       0.81      0.44      0.57      1090

avg / total       0.88      0.89      0.88      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.71      0.87      0.78      3879
          1       0.71      0.48      0.57      2675

avg / total       0.71      0.71      0.70      6554

medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0

In [15]:
#view avg for all targets
print("precision    recall  f1-score   support")
print(metrics.precision_recall_fscore_support(y_test, y_pred2, average='macro'))

precision    recall  f1-score   support
(0.4494717412014384, 0.16051565043337238, 0.20367835091377909, None)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline2 = Pipeline([
            ('vect', CountVectorizer(stop_words={'english'}, tokenizer=tokenize, min_df=0.01))
             ,('tfidf', TfidfTransformer())
             ,('clf', MultiOutputClassifier(LogisticRegression(random_state=0), n_jobs=-1))
            ])

pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0.01,
        ngram_range=(1, 1), preprocessor=None, stop_words={'english'},
   ...e=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=-1))])

In [17]:
y_pred3 = pipeline2.predict(X_test)

for i in range(len(y_test.columns)):
    print(y_test.columns[i],metrics.classification_report(y_test.iloc[:,i], y_pred3[:,i]))

related              precision    recall  f1-score   support

          0       0.64      0.40      0.50      1544
          1       0.84      0.93      0.88      5010

avg / total       0.79      0.81      0.79      6554

request              precision    recall  f1-score   support

          0       0.92      0.97      0.94      5464
          1       0.81      0.55      0.65      1090

avg / total       0.90      0.90      0.90      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.75      0.84      0.79      3879
          1       0.72      0.58      0.64      2675

avg / total       0.73      0.74      0.73      6554

medical_help              precision    recall  f1-score   support

          0       0.93      0.99      0

In [18]:
#view avg for all targets
print("precision    recall  f1-score   support")
print(metrics.precision_recall_fscore_support(y_test, y_pred3, average='macro'))

precision    recall  f1-score   support
(0.50053258987703053, 0.20284473350185273, 0.25176412471847942, None)


In [19]:
#refit best model on entire dataset and save as final_model
#the logistic regression performed better than the random forest
final_model = pipeline2.fit(X, y)

### 9. Export your model as a pickle file

In [20]:
filename = 'finalized_classifier.sav'
pickle.dump(final_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.